In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [2]:
pip install catboost

  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/1c/e1/78e635a1e5f0066bd02a1ecfd658ad09fe30d275c65c2d0dd76fe253e648/catboost-1.2.7-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for graphviz from https://files.pythonhosted.org/packages/00/be/d59db2d1d52697c6adc9eacaf50e8965b6345cc143f671e1ed068818d5cf/graphviz-0.20.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB 487.6 kB/s eta 0:03:29
   ---------------------------------------- 0.1/101.7 MB 819.2 kB/s eta 0:02:05
   ---------------------------------------- 0.2/101.7 MB 952.6 kB/s eta 0:01:47
   ---------------------------------------- 0.3/101.7 MB 1.2 MB/s eta 0:01:24
   ---------------------------------------- 0.3/101.7 MB 

In [4]:
train_dir = r'D:\Computer Vision\Alzheimer_s Dataset\Alzheimer_s Dataset\train'  
test_dir = r'D:\Computer Vision\Alzheimer_s Dataset\Alzheimer_s Dataset\test'

In [5]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [6]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [7]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [8]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [9]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

D:\anacondafiles\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

D:\anacondafiles\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 18s 107ms/step


In [11]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 100ms/step


In [12]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3239864	total: 191ms	remaining: 3m 10s
1:	learn: 1.2731984	total: 239ms	remaining: 1m 59s
2:	learn: 1.2315933	total: 290ms	remaining: 1m 36s
3:	learn: 1.1937190	total: 339ms	remaining: 1m 24s
4:	learn: 1.1622486	total: 386ms	remaining: 1m 16s
5:	learn: 1.1344498	total: 443ms	remaining: 1m 13s
6:	learn: 1.1095477	total: 494ms	remaining: 1m 10s
7:	learn: 1.0884444	total: 539ms	remaining: 1m 6s
8:	learn: 1.0677002	total: 568ms	remaining: 1m 2s
9:	learn: 1.0490956	total: 594ms	remaining: 58.8s
10:	learn: 1.0339116	total: 625ms	remaining: 56.2s
11:	learn: 1.0192634	total: 651ms	remaining: 53.6s
12:	learn: 1.0052435	total: 675ms	remaining: 51.3s
13:	learn: 0.9938490	total: 697ms	remaining: 49.1s
14:	learn: 0.9833212	total: 720ms	remaining: 47.3s
15:	learn: 0.9725965	total: 743ms	remaining: 45.7s
16:	learn: 0.9625706	total: 766ms	remaining: 44.3s
17:	learn: 0.9526117	total: 789ms	remaining: 43s
18:	learn: 0.9446871	total: 812ms	remaining: 41.9s
19:	learn: 0.9372980	total: 837ms	re

161:	learn: 0.5633959	total: 4.78s	remaining: 24.7s
162:	learn: 0.5618819	total: 4.8s	remaining: 24.6s
163:	learn: 0.5604985	total: 4.82s	remaining: 24.6s
164:	learn: 0.5574942	total: 4.85s	remaining: 24.5s
165:	learn: 0.5565349	total: 4.87s	remaining: 24.5s
166:	learn: 0.5539774	total: 4.89s	remaining: 24.4s
167:	learn: 0.5528050	total: 4.92s	remaining: 24.3s
168:	learn: 0.5514972	total: 4.94s	remaining: 24.3s
169:	learn: 0.5490476	total: 4.96s	remaining: 24.2s
170:	learn: 0.5477031	total: 4.98s	remaining: 24.2s
171:	learn: 0.5457766	total: 5s	remaining: 24.1s
172:	learn: 0.5440097	total: 5.03s	remaining: 24s
173:	learn: 0.5422945	total: 5.05s	remaining: 24s
174:	learn: 0.5403323	total: 5.07s	remaining: 23.9s
175:	learn: 0.5389030	total: 5.09s	remaining: 23.9s
176:	learn: 0.5366245	total: 5.12s	remaining: 23.8s
177:	learn: 0.5348468	total: 5.14s	remaining: 23.7s
178:	learn: 0.5330086	total: 5.17s	remaining: 23.7s
179:	learn: 0.5314894	total: 5.19s	remaining: 23.6s
180:	learn: 0.530316

323:	learn: 0.3652174	total: 8.72s	remaining: 18.2s
324:	learn: 0.3645798	total: 8.74s	remaining: 18.2s
325:	learn: 0.3630743	total: 8.77s	remaining: 18.1s
326:	learn: 0.3618052	total: 8.79s	remaining: 18.1s
327:	learn: 0.3608201	total: 8.81s	remaining: 18.1s
328:	learn: 0.3599726	total: 8.84s	remaining: 18s
329:	learn: 0.3586953	total: 8.86s	remaining: 18s
330:	learn: 0.3581806	total: 8.89s	remaining: 18s
331:	learn: 0.3578185	total: 8.93s	remaining: 18s
332:	learn: 0.3569022	total: 8.97s	remaining: 18s
333:	learn: 0.3558928	total: 9.02s	remaining: 18s
334:	learn: 0.3547311	total: 9.06s	remaining: 18s
335:	learn: 0.3538175	total: 9.09s	remaining: 18s
336:	learn: 0.3534234	total: 9.11s	remaining: 17.9s
337:	learn: 0.3526162	total: 9.13s	remaining: 17.9s
338:	learn: 0.3517488	total: 9.16s	remaining: 17.9s
339:	learn: 0.3509875	total: 9.18s	remaining: 17.8s
340:	learn: 0.3500857	total: 9.21s	remaining: 17.8s
341:	learn: 0.3495854	total: 9.23s	remaining: 17.8s
342:	learn: 0.3490375	total:

488:	learn: 0.2490967	total: 13.9s	remaining: 14.5s
489:	learn: 0.2483768	total: 13.9s	remaining: 14.5s
490:	learn: 0.2475768	total: 13.9s	remaining: 14.5s
491:	learn: 0.2472861	total: 14s	remaining: 14.4s
492:	learn: 0.2470095	total: 14s	remaining: 14.4s
493:	learn: 0.2466015	total: 14s	remaining: 14.4s
494:	learn: 0.2460318	total: 14.1s	remaining: 14.3s
495:	learn: 0.2451265	total: 14.1s	remaining: 14.3s
496:	learn: 0.2444729	total: 14.1s	remaining: 14.3s
497:	learn: 0.2437852	total: 14.2s	remaining: 14.3s
498:	learn: 0.2432207	total: 14.2s	remaining: 14.2s
499:	learn: 0.2427987	total: 14.2s	remaining: 14.2s
500:	learn: 0.2423852	total: 14.3s	remaining: 14.2s
501:	learn: 0.2418735	total: 14.3s	remaining: 14.2s
502:	learn: 0.2415590	total: 14.4s	remaining: 14.3s
503:	learn: 0.2409891	total: 14.5s	remaining: 14.2s
504:	learn: 0.2405715	total: 14.5s	remaining: 14.2s
505:	learn: 0.2398417	total: 14.5s	remaining: 14.2s
506:	learn: 0.2391747	total: 14.6s	remaining: 14.2s
507:	learn: 0.2386

648:	learn: 0.1784277	total: 17.8s	remaining: 9.64s
649:	learn: 0.1780799	total: 17.8s	remaining: 9.61s
650:	learn: 0.1774837	total: 17.9s	remaining: 9.58s
651:	learn: 0.1770923	total: 17.9s	remaining: 9.55s
652:	learn: 0.1765982	total: 17.9s	remaining: 9.52s
653:	learn: 0.1762799	total: 17.9s	remaining: 9.49s
654:	learn: 0.1760101	total: 18s	remaining: 9.46s
655:	learn: 0.1755285	total: 18s	remaining: 9.43s
656:	learn: 0.1750897	total: 18s	remaining: 9.4s
657:	learn: 0.1745925	total: 18s	remaining: 9.37s
658:	learn: 0.1740309	total: 18s	remaining: 9.34s
659:	learn: 0.1735369	total: 18.1s	remaining: 9.31s
660:	learn: 0.1731222	total: 18.1s	remaining: 9.28s
661:	learn: 0.1726913	total: 18.1s	remaining: 9.25s
662:	learn: 0.1722830	total: 18.1s	remaining: 9.22s
663:	learn: 0.1718786	total: 18.2s	remaining: 9.19s
664:	learn: 0.1713609	total: 18.2s	remaining: 9.16s
665:	learn: 0.1710711	total: 18.2s	remaining: 9.13s
666:	learn: 0.1704890	total: 18.2s	remaining: 9.1s
667:	learn: 0.1702356	to

814:	learn: 0.1298572	total: 22.4s	remaining: 5.08s
815:	learn: 0.1296547	total: 22.4s	remaining: 5.05s
816:	learn: 0.1292426	total: 22.4s	remaining: 5.03s
817:	learn: 0.1290145	total: 22.5s	remaining: 5s
818:	learn: 0.1287263	total: 22.5s	remaining: 4.97s
819:	learn: 0.1285227	total: 22.5s	remaining: 4.94s
820:	learn: 0.1281199	total: 22.6s	remaining: 4.92s
821:	learn: 0.1279550	total: 22.6s	remaining: 4.89s
822:	learn: 0.1277889	total: 22.6s	remaining: 4.86s
823:	learn: 0.1276720	total: 22.6s	remaining: 4.84s
824:	learn: 0.1273643	total: 22.7s	remaining: 4.81s
825:	learn: 0.1271451	total: 22.7s	remaining: 4.78s
826:	learn: 0.1270570	total: 22.7s	remaining: 4.75s
827:	learn: 0.1268986	total: 22.8s	remaining: 4.73s
828:	learn: 0.1267206	total: 22.8s	remaining: 4.7s
829:	learn: 0.1265174	total: 22.8s	remaining: 4.67s
830:	learn: 0.1261726	total: 22.8s	remaining: 4.64s
831:	learn: 0.1258930	total: 22.9s	remaining: 4.62s
832:	learn: 0.1255301	total: 22.9s	remaining: 4.59s
833:	learn: 0.12

980:	learn: 0.0976252	total: 27s	remaining: 524ms
981:	learn: 0.0975488	total: 27.1s	remaining: 496ms
982:	learn: 0.0974000	total: 27.1s	remaining: 468ms
983:	learn: 0.0972494	total: 27.1s	remaining: 441ms
984:	learn: 0.0971571	total: 27.1s	remaining: 413ms
985:	learn: 0.0970065	total: 27.2s	remaining: 386ms
986:	learn: 0.0968085	total: 27.2s	remaining: 358ms
987:	learn: 0.0966685	total: 27.2s	remaining: 330ms
988:	learn: 0.0965042	total: 27.2s	remaining: 303ms
989:	learn: 0.0963724	total: 27.2s	remaining: 275ms
990:	learn: 0.0962185	total: 27.3s	remaining: 248ms
991:	learn: 0.0960904	total: 27.3s	remaining: 220ms
992:	learn: 0.0959410	total: 27.3s	remaining: 193ms
993:	learn: 0.0958416	total: 27.3s	remaining: 165ms
994:	learn: 0.0956499	total: 27.4s	remaining: 137ms
995:	learn: 0.0954811	total: 27.4s	remaining: 110ms
996:	learn: 0.0953126	total: 27.4s	remaining: 82.5ms
997:	learn: 0.0951641	total: 27.4s	remaining: 55ms
998:	learn: 0.0949758	total: 27.4s	remaining: 27.5ms
999:	learn: 0

In [22]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [23]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 49.96%


In [15]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [16]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [19]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model('cnn_feature_extractor6.h5')
catboost_classifier = joblib.load('catboost_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = r'D:\Computer Vision\Alzheimer_s Dataset\Alzheimer_s Dataset\test\VeryMildDemented\26 (44).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
The image is classified as: VeryMildDemented


C:\Users\ELMAJD COMP\AppData\Local\Temp\ipykernel_23452\1548775328.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(prediction[0])
